# Dashboard

# ESMR data from open data
https://data.ca.gov/dataset/water-quality-effluent-electronic-self-monitoring-report-esmr-data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from holoviews import opts, dim
import holoviews as hv
import panel as pn
import hvplot.pandas
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
pn.extension()
hv.extension('bokeh')

In [ ]:
import esmr

In [ ]:
df = esmr.read_data_csv(
    'esmr-analytical-export_years-2006-2022_2022-11-17/esmr-analytical-export_years-2006-2022_2022-11-17.csv')

In [ ]:
data_dict = pd.read_csv('esmr_data_dictionary.csv')

In [ ]:
data = esmr.ESMR(df)

## Plot for facilities that have location information of lat/lon

In [ ]:
facility_location_lat_lon = esmr.build_facility_location_lat_lon(df)

In [ ]:
facility_location_lat_lon.hvplot.points(x='longitude', y='latitude', geo=True, hover_cols='all', tiles=True).opts(
    frame_width=500, active_tools=['wheel_zoom', 'pan'])

# Facilities with no location information

In [ ]:
df_facility_lat_lon = esmr.build_facility_location_lat_lon(df)

In [ ]:
df_facility_lat_lon = df_facility_lat_lon.groupby(['facility_name']).mean().dropna().reset_index()

In [ ]:
df_facility_lat_lon

In [ ]:
#esmr.get_facilities_with_no_latlon_info(df, facility_location_lat_lon) # facilities that need location information

In [ ]:
# map of facilities
map = df_facility_lat_lon.hvplot.points(
    x='longitude', y='latitude', geo=True, tiles='CartoLight', hover_cols='all').opts(frame_width=400, aspect=1)
select_map_pts = hv.streams.Selection1D(source=map.Points.I)


def show_plot(index):
    if len(index) == 0:
        index = [0]
    facility_name = df_facility_lat_lon.iloc[index[0]].facility_name
    # .opts(title=f'{index[0]} - {facility_name}')
    plot = plot_parameters(data, facility_name)
    return plot.cols(1)
    #return hv.Layout([plot.cols(1)]).cols(1)  # .opts(responsive=True)


# connect selection to plot
# .opts(responsive=True)
dmap = hv.DynamicMap(show_plot, streams=[select_map_pts], cache_size=1)

In [ ]:
pn.Row(map.opts(opts.Points(tools=['tap', 'hover'], active_tools=['wheel_zoom', 'pan', 'tap'],
                            nonselection_color='gray', nonselection_alpha=0.5, size=8)).opts(frame_height=1000, frame_width=600), pn.panel(dmap)).show()